In [5]:
import os
import pandas as pd
import plotly.express as px
import toml

def plot_job_id(job_id: int, mode: str = 'period'):
    directory = os.path.join(os.getcwd(), f'{job_id}')
    cf = toml.load(os.path.join(directory, 'config.toml'))
    df = pd.read_csv(os.path.join(directory, 'results.csv'))

    print(cf['model']['name'], cf['timing']['path'].split('/')[-1])

    # Display the combined dataframe
    # print(combined_df)

    if mode == 'period':
        df_opt = df.loc[df.groupby('period')["p99"].idxmin()]
        return px.line(
            df_opt.loc[df_opt['hit_chance'] > 0.3],
            x='period', 
            y='p99',
        ).update_layout(
            xaxis_title="Percentile of Execution Time P (P = T)",
            yaxis_title="99-th Percentile of Safety Property D"
        )
    elif mode == 'util':
        df_opt = df.loc[df.groupby('utilization')["p99"].idxmin()]
        return px.line(
            df_opt.loc[df_opt['hit_chance'] > 0.3],
            x='utilization', 
            y='p99',
        ).update_layout(
            xaxis_title="Utilization",
            yaxis_title="99-th Percentile of safety property D"
        )
    elif mode == 'clock':
        df_opt = df.loc[df.groupby('clock_speed')["p99"].idxmin()]
        df_wcet = df.loc[df.groupby('clock_speed')["hit_chance"].idxmax()]
        # df_opt = df
        return px.line(
            # df_opt.loc[df_opt['clock_speed'] <= 2_000_000],
            df_opt.loc[df_opt['p99'] <= 10],
            # df_opt,
            x='clock_speed', 
            y='p99',
            hover_data=['clock_speed', 'period', 'hit_chance']
        ).update_layout(
            xaxis_title="Clock Speed",
            yaxis_title="99-th Percentile of safety property D"
        ).add_scatter(
            x=df.loc[(df['hit_chance'] >= 0.999)]['clock_speed'],
            y=df.loc[(df['hit_chance'] >= 0.999)]['p99'],
            mode='lines',
            name='99.9% hit chance',
        )
        # , px.line(
        #     # df.loc[(df['hit_chance'] == 1) & (df['clock_speed'] <= 2_000_000)],
        #     df.loc[(df['hit_chance'] == 1)],
        #     x='clock_speed',
        #     y='p99',
        #     hover_data=['clock_speed', 'period', 'hit_chance']
        # ).update_layout(
        #     xaxis_title="Clock Speed",
        #     yaxis_title="99-th Percentile of safety property D"
        # )
        

In [6]:
plot_job_id('clock_speeds_20241111_0329', mode='clock')

CC huffbench.json


In [7]:
plot_job_id('clock_speeds_20241111_0631', mode='clock')

F1 picojpeg-emnist.json


In [8]:
plot_job_id('clock_speeds_20241111_0649', mode='clock')

F1 picojpeg-emnist.json
